<a href="https://colab.research.google.com/github/raydavies/capstone-sentiment-analysis-app-starter/blob/main/notebooks/uci_sentiment_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download dataset from the UCI website
!curl -o uci-labelled-sentences.zip https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip

# unzip dataset in Colab
!unzip uci-labelled-sentences.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84188    0 84188    0     0   335k      0 --:--:-- --:--:-- --:--:--  336k
Archive:  uci-labelled-sentences.zip
   creating: sentiment labelled sentences/
  inflating: sentiment labelled sentences/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/sentiment labelled sentences/
  inflating: __MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentiment labelled sentences/imdb_labelled.txt  
  inflating: __MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentiment labelled sentences/readme.txt  
  inflating: __MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentiment labelled sentences/yelp_labelled.txt  
  inflating: __MACOSX/._sentiment labelled sentences  


In [9]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Input, LSTM
from keras.callbacks import EarlyStopping

In [5]:
df_list = []

# Yelp
df_yelp = pd.read_csv('sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df_yelp['source'] = 'yelp'
df_list.append(df_yelp)

# Amazon
df_amazon = pd.read_csv('sentiment labelled sentences/amazon_cells_labelled.txt', names=['sentence', 'label'], sep='\t')
df_amazon['source'] = 'amazon'
df_list.append(df_amazon)

# IMDB
df_imdb = pd.read_csv('sentiment labelled sentences/imdb_labelled.txt', names=['sentence', 'label'], sep='\t')
df_imdb['source'] = 'imdb'
df_list.append(df_imdb)

# Concatenate the dataframes
df = pd.concat(df_list)

df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [6]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['sentence'].values)
X = tokenizer.texts_to_sequences(df['sentence'].values)
X = pad_sequences(X)
y = df['label'].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12)

In [11]:
def create_model():
  model = Sequential()
  model.add(Input(shape=(X.shape[1],)))
  model.add(Embedding(max_features, 64))
  model.add(LSTM(16))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = create_model()

In [12]:
model.fit(X_train, y_train, epochs=6, batch_size=16, validation_data=(X_test, y_test), callbacks = [EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2, verbose=1)])

Epoch 1/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 77s 485ms/step - accuracy: 0.5534 - loss: 0.6794 - val_accuracy: 0.8152 - val_loss: 0.4845
Epoch 2/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 80s 474ms/step - accuracy: 0.8514 - loss: 0.3919 - val_accuracy: 0.8515 - val_loss: 0.3753
Epoch 3/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 72s 472ms/step - accuracy: 0.9325 - loss: 0.2053 - val_accuracy: 0.8424 - val_loss: 0.3932
Epoch 4/6
152/152 ━━━━━━━━━━━━━━━━━━━━ 72s 473ms/step - accuracy: 0.9532 - loss: 0.1452 - val_accuracy: 0.8455 - val_loss: 0.4534
Epoch 4: early stopping


In [13]:
model.save("uci_sentimentanalysis.h5")

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.DEFAULT_PROTOCOL)